# How to use "iemap-mi" Python module

1) Install the module using pip    <br/><br/>
   ```pip install -U iemap-mi```    <br/><br/>

2) Or install all modules using poetry : <br/> <br/>
   ```poetry install```<br><br/>    
   This will install all the required modules including iemap-mi, according to .toml file <br/><br/>

**If using the Docker image, all the modules are already installed.** <br/><br/>
Uncomment cell below and run it to install module (if not yet installed)

In [ ]:
# !pip install -U iemap-mi

The ```%autoawait on``` magic command is a feature specific to IPython and Jupyter environments   
that simplifies working with asynchronous code.    
Here's a detailed explanation:   

**Purpose**:   

```%autoawait on``` allows you to use await expressions directly in the notebook without    
explicitly creating and running a coroutine or event loop.

**How it works**:   

When enabled, IPython automatically wraps each cell's contents in an async function.
It then runs this function using the current event loop.

**Benefits**:

Simplifies async code execution in notebooks.
Allows you to write async code more naturally, as if you were in an async function.
Eliminates the need for boilerplate code to run async functions.

Usage:

Run ```%autoawait on``` at the beginning of your notebook or in any cell before you start using async code.
After enabling, you can use await directly in your cells.

In [ ]:
%autoawait on

## Import necessary modules

In [ ]:
import json
import httpx
import pandas as pd
from pprint import pprint
from pathlib import Path
from pydantic import TypeAdapter
from tqdm import tqdm
from iemap_mi import IemapMI
from ipywidgets import widgets, FileUpload
from typing import List
from iemap_mi.models import (IEMAPProject, Project, Material, Process, Agent,
                             Parameter, Property, FlattenedProjectBase, FlattenedProjectHashEmail, FileInfo)
from iemap_mi.project_handler import ProjectHandler
from iemap_mi.utils import flatten_project_data

3. Instantiate the class, call it "client" (or anyway you want) and start using it.

In [ ]:
# Initialize the client
client = IemapMI()

4. Start by printing the version of the module.

In [ ]:
# Print the module version (for time of writing this is 0.2.0)
IemapMI.print_version()

In [ ]:
# Get user credentials, necessary for authentication and authorization of the user with the API
username_widget = widgets.Text(description="Username:", placeholder='Enter your username (email address)',)
password_widget = widgets.Password(description="Password:",    placeholder='Enter your password',)
display(username_widget, password_widget)

In [ ]:
# Get the input values from the widgets
username = username_widget.value
password = password_widget.value
# Check if the username and password are valid
if username=="" or password=="":
    raise ValueError("Please enter a valid username and password.")

In [ ]:
# Authenticate the user with the API
# This function will authenticate the user with the API using the provided username and password
# If successful, it will return the user's token (which can be used for subsequent API calls)
# If unsuccessful, it will raise an exception with the error message
# JWT is stored in the client object (client.token)
async def authenticate_user(client_iemap, user_email, passwd):
    await client_iemap.authenticate(username=user_email, password=passwd)
    print("Authenticated successfully!")

In [ ]:
# Authenticate the user
# This will set the client.token if successful
# If unsuccessful, it will raise an exception
# being an async function, we need to use await, so we use the await keyword
# to call the authenticate_user function, this works only in an async environment, this is why we use %autoawait on
result_auth=await authenticate_user(client, username, password) 

In [ ]:
# JWT can be shown using client.token

## Example of using the API to fetch projects   

1. define an helper async function, *iterate_projects*,  to fetch projects 
2. use method get_projects from project_handler to fetch projects
3. flatten the projects data and display it in a pandas dataframe


In [ ]:
# Example of how to get the projects
async def iterate_projects(client: IemapMI, page_size: int = 40) -> None:
    page_number = 1
    all_projects: List[FlattenedProjectBase] = []
    total_projects = None

    while True:
        projects_response = await client.project_handler.get_projects(page_size=page_size, page_number=page_number)
        if not projects_response.data:
            break

        adapter = TypeAdapter(FlattenedProjectHashEmail)
        projects = [adapter.validate_python(project) for project in projects_response.data]

        all_projects.extend(projects)
        page_number += 1

        if total_projects is None:
            total_projects = projects_response.number_docs

        tqdm.write(f"Page {page_number - 1} fetched. Total projects so far: {len(all_projects)}/{total_projects}")

    
    pd.set_option('display.max_columns', None)
    flat_projects = [flatten_project_data(project) for project in all_projects]
    df = pd.DataFrame(flat_projects)
    display(df)
 

In [ ]:
# Fetch and display projects
await iterate_projects(client, page_size=60)

## Example of using the API to get projects statistics

In [ ]:
# Fetch statistics data
stats = await client.stat_handler.get_stats()
print(stats.model_dump())

## Example of using the API to query projects

In [ ]:
# Query projects
query_response = await client.project_handler.query_projects(
    isExperiment=True,
    limit=10
)

print(f"Found {len(query_response)} projects:")
print("-" * 40)
# pretty print the projects
for index, doc in enumerate(query_response, start=1):
    print(f"Project {index}:")
    # Convert to dict and then to JSON for pretty printing
    project_dict = json.loads(doc.model_dump_json())
    # Pretty print with custom formatting
    pprint(project_dict, width=100, sort_dicts=False, indent=2)
    print("-" * 40)  # Separator between projects

## Example of using the API to create a new project

1. Define data as a dictionary
2. validate schema using pydantic model
3. insert the project using the project_handler.create_project method

In [ ]:
data = {
    "project": {
        "name": "Materials for Batteries",
        "label": "MB",
        "description": "IEMAP - TEST Project for batteries"
    },
    "material": {
        "formula": "C15H20N2F6S2O4"
    },
    "process": {
        "method": "Karl-Fischer titration",
        "agent": {
            "name": "Karl-Fischer titrator Mettler Toledo",
            "version": None
        },
        "isExperiment": True
    },
    "parameters": [
        {
            "name": "time",
            "value": 20,
            "unit": "s"
        },
        {
            "name": "weight",
            "value": 0.5,
            "unit": "gr"
        }
    ],
    "properties": [
        {
            "name": "Moisture content",
            "value": "<2",
            "unit": "ppm"
        }
    ]
}

In [ ]:
valid_payload = ProjectHandler.build_project_payload(data)
if valid_payload:
    print("Payload is valid and ready to be submitted.")
    current_proj = IEMAPProject(**valid_payload)
    new_project = await client.project_handler.create_project(current_proj)
    print(new_project)
else:
    print("Payload is invalid.")

## Add a file to the project
1. This is a two step process, first we define a project or get a valid project id
2. Then we define a file object and add it to the project
3. The file object is a dictionary containing the file content and the file name
4. We use the *project_handler.add_file_to_project* method to add the file to the project

> For example, we can use the project created in the previous step: new_project.inserted_id

In [ ]:
new_project.inserted_id

In [ ]:
fw = FileUpload(description="Choose file:",  
                         accept='', # Accepted file extension e.g. '.txt', '.pdf', 'image/', 'image/,.pdf'
                         multiple=False # True to accept multiple files upload else False
 )
fw

In [ ]:
# Upload the file to the project
# path_files='./host_files'
if fw.value:
    file_name = fw.value[0]['name'].split(".")[0]
    file_full_path = fw.value[0]['name'] #  path_files +'/'+fw.value[0]['name'] if executing from Docker container
    file_response = await client.project_handler.add_file_to_project(
        project_id=new_project.inserted_id, # Use the project ID from the previous step or any valid project ID
        file_path=file_full_path,
        file_name=file_name
    )
    if file_response['uploaded']:
        file_info = FileInfo(**file_response)
        print(f"File {file_info.file_name} uploaded successfully to project (saved as {file_info.file_hash})")
    else:
        print("File upload failed.")

## Add file to Project using a file dialog in case Ipywidgets are not working

In [ ]:
import tkinter as tk
from tkinter import filedialog
from pathlib import Path

async def select_and_upload_file(client, project_id):
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path_to_upload = filedialog.askopenfilename(filetypes=[("All files", "*.*")], initialdir="./")

    if not file_path_to_upload:  # User cancelled file selection
        print("File selection cancelled.")
        return

    path = Path(file_path_to_upload)

    if path.exists() and path.is_file():
        file_name = path.name
        try:
            # with open(path, 'rb') as file:
            #     file_content = file.read()

            file_response = await client.project_handler.add_file_to_project(
                project_id= project_id,
                file_path=file_path_to_upload,
                file_name=file_name
            )

            if file_response['uploaded']:
                file_info = FileInfo(**file_response)
                print(f"File {file_info.file_name} uploaded successfully to project (saved as {file_info.file_hash})")
            else:
                print("File upload failed.")
        except IOError as e:
            print(f"Error reading file: {e}")
        except Exception as e:
            print(f"An error occurred during file upload: {e}")
    elif path.exists() and not path.is_file():
        print("The selected path is not a file.")
    else:
        print("The selected file does not exist.")


In [ ]:
await select_and_upload_file(client, "66d5b9d5b995c52cd95b1dc2")#, new_project.inserted_id)

In [ ]:
client.token

## Example of file download using REST API instead of iemap-mi module

In [ ]:
import httpx
async def retrieve_file(file_hash: str, file_extension: str, jwt_token: str, save_path: str = None):
    base_url = "https://iemap.enea.it/rest"
    file_url = f"{base_url}/file/{file_hash}{file_extension}"
    headers = {
        "Authorization": f"Bearer {jwt_token}"
    }

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(file_url, headers=headers)
            response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)

            if save_path:
                # If a save path is provided, save the file
                file_path = Path(save_path)
                file_path.write_bytes(response.content)
                print(f"File saved successfully to {file_path}")
                return file_path
            else:
                # If no save path is provided, return the content
                return response.content

        except httpx.HTTPStatusError as e:
            print(f"HTTP error occurred: {e}")
        except httpx.RequestError as e:
            print(f"An error occurred while requesting {e.request.url!r}.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [ ]:
# Download the file
hash_file = "HASH_FILE"
await retrieve_file(hash_file, ".pdf", client.token, save_path="./downloaded_file.pdf")